In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from funkwpap import sympy_to_gurobi, Firm, Country, Sector, Regulator, get_emission
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys
from gurobipy import Model, LinExpr, QuadExpr, GRB
x = sp.symbols('x')
y = sp.symbols('y')

In [2]:

# Create Sectors and Countries
Regulator1 = Regulator('EU', permit_price= 2, emission_cap= 100)
sector1 = Sector('cement', price_demand_function= 100 - 0.1*x, free_emission_multiplier= 0.1, regulator= Regulator1)
sector2 = Sector('steel', price_demand_function=150 - 0.1*x, free_emission_multiplier= 0.05, regulator= Regulator1)
sector3 = Sector('paper', price_demand_function= 200 - 0.02*x**1.5, free_emission_multiplier= 0.1, regulator= Regulator1)
country1 = Country('DE', 1, regulator= Regulator1)
country2 = Country('FI', 0.5, regulator= Regulator1)
country3 = Country('GR', size= 0.1, regulator= Regulator1)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 0.1*x**4 + 100/y**2, 0, 0, 0, regulator= Regulator1)
firm2 = Firm('firm2', 1, 2, x*0, 5*x**3.2 + 40/y**2, 0, 0, 0, regulator= Regulator1)
firm3 = Firm('firm3', 1, 3, x*0, x**5.2 + 30/y**2, 0, 0, 0, regulator= Regulator1)
firm4 = Firm('firm4', 2, 1, x*0, x**3.3 + 20/y**2, 0, 0, 0, regulator= Regulator1)
firm5 = Firm('firm5', 2, 2, x*0, x**2.3 + 20/y**2, 0, 0, 0, regulator= Regulator1)
firm6 = Firm('firm6', 2, 3, x*0, x**3.2 + 15/y**2, 0, 0, 0, regulator= Regulator1)
firm7 = Firm('firm7', 3, 1, x*0, x**4.1 + 40/y**2, 0, 0, 0, regulator= Regulator1)
firm8 = Firm('firm8', 3, 2, x*0, x**5.1 + 50/y**2, 0, 0, 0, regulator= Regulator1)
firm9 = Firm('firm9', 3, 3, x*0, x**5 + 60/y**2, 0, 0, 0, regulator= Regulator1)


print(sector1.firms)  # List of firms in sector1
print(country1.firms)  # List of firms in country1
print(Regulator1.firm_registry)  # Dictionary of all firms

for firm in Regulator1.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name)


[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=2, name='firm2', sector_id=1, country_id=2, actual_output=0, emission=0, profit=0), Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0)]
[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=7, name='firm7', sector_id=3, country_id=1, actual_output=0, emission=0, profit=0)]
{1: Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), 2: Firm(id=2, name='firm2', sector_id=1, country_id=2, actual_output=0, emission=0, profit=0), 3: Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0), 4: Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output=0, emission=0, profit=0), 5: Firm(id=5, name='firm5', sector_id=2, country_id=2, actual_out

In [3]:
Regulator1.optimize_them_all(precision = 0.1, print_diff = True, BAU = True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-14
Max diff: 0.06, permit price = 2.0, cap = 100, second_stage = False, a = 1.0000

In [4]:
for firm in Regulator1.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name, firm.actual_output, firm.emission, firm.profit)

firm1 cement DE 257.32525385607016 0.0 6273.59076591368
firm2 cement FI 257.32525385607016 0.0 6273.59076591368
firm3 cement GR 241.54945043799546 0.0 5888.97661255950
firm4 steel DE 376.13222749306016 0.0 13977.1983562917
firm5 steel FI 376.13222749306016 0.0 13977.1983562917
firm6 steel GR 376.13222749306016 0.0 13977.1983562917
firm7 paper DE 118.59141288110008 0.0 7795.70908896106
firm8 paper FI 118.55901246575188 0.0 7794.93391399515
firm9 paper GR 118.67118725249841 0.0 7803.94427937029


In [5]:
Regulator1.BAU_calculator(print_diff = True)

Max diff: 0.01, permit price = 2.0, cap = 100, second_stage = False, a = 0.73

In [6]:
print("Firm name, Sector, Country, Actual output, Emission, Profit, BAU profit, BAU emission")
for firm in Regulator1.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name, firm.actual_output, firm.emission, firm.profit, firm.BAU_profit, firm.BAU_emission)

Firm name, Sector, Country, Actual output, Emission, Profit, BAU profit, BAU emission
firm1 cement DE 257.32525385607016 0.0 6273.59076591368 6273.59076591368 257.32525385607016
firm2 cement FI 257.32525385607016 0.0 6273.59076591368 6273.59076591368 257.32525385607016
firm3 cement GR 241.54945043799546 0.0 5888.97661255950 5888.97661255950 241.54945043799546
firm4 steel DE 376.1322274930602 0.0 13977.1983562917 13977.1983562917 376.1322274930602
firm5 steel FI 376.1322274930602 0.0 13977.1983562917 13977.1983562917 376.1322274930602
firm6 steel GR 376.1322274930602 0.0 13977.1983562917 13977.1983562917 376.1322274930602
firm7 paper DE 118.63237526213813 0.0 7800.03307363717 7800.03307363717 118.63237526213813
firm8 paper FI 118.61365635541384 0.0 7798.35734205865 7798.35734205865 118.61365635541384
firm9 paper GR 118.6044563033629 0.0 7797.61789018947 7797.61789018947 118.6044563033629


|    |   Firm |     Output |   Emission |      Profit |   sector |   sum_sector_output |     price |      sales |   abatement |   trades |         sum | Correct? |
|---:|-------:|-----------:|-----------:|------------:|---------:|--------------------:|----------:|-----------:|------------:|---------:|------------:|:---------|
|  0 |      1 | 257.325    |    257.325 |   6273.5908 |        1 |            756.2    |  24.38    |   6273.59  |  0.0015102  |        0 |   6273.5893 | TRUE      |
|  1 |      2 | 257.325    |    257.325 |   6273.5908 |        1 |            756.2    |  24.38    |   6273.59  |  0.0006041  |        0 |   6273.5902 | TRUE      |
|  2 |      3 | 241.549    |    241.549 |   5888.9766 |        1 |            756.2    |  24.38    |   5888.98  |  0.0005142  |        0 |   5888.9761 | TRUE      |
|  3 |      4 | 376.132    |    376.132 |  13977.1984 |        2 |           1128.4    |  37.1603  |  13977.2   |  0.0001414  |        0 |  13977.1982 | TRUE      |
|  4 |      5 | 376.132    |    376.132 |  13977.1984 |        2 |           1128.4    |  37.1603  |  13977.2   |  0.0001414  |        0 |  13977.1982 | TRUE      |
|  5 |      6 | 376.132    |    376.132 |  13977.1984 |        2 |           1128.4    |  37.1603  |  13977.2   |  0.000106   |        0 |  13977.1982 | TRUE      |
|  6 |      7 | 118.455    |    118.455 |   7789.6433 |        3 |            355.366  |  66.0189  |   7820.29  |  0.0028507  |        0 |   7820.2833 | TRUE      |
|  7 |      8 | 118.455    |    118.455 |   7789.6433 |        3 |            355.366  |  66.0189  |   7820.29  |  0.0035634  |        0 |   7820.2826 | TRUE      |
|  8 |      9 | 118.455    |    118.455 |   7789.6433 |        3 |            355.366  |  66.0189  |   7820.29  |  0.004276   |        0 |   7820.2819 | TRUE      |


In [7]:
# Create a plot for the permit price with respect to the emission cap 10 samples between 0 and 200
BAU_percentage = np.linspace(0.2, 1, 20)
BAU_emissions = Regulator1.BAU_emissions
caps = [BAU_emissions*i for i in BAU_percentage]
# Read the data from the csv file, if it exists, if not, just create a new one
try:
    df = pd.read_csv('permit_price_to_Percentage_of_BAU_v0_1.csv')
    percentages = list(df['Percentage of BAU'])
    caps2 = list(df['Emission Cap'])
    permit_prices = list(df['Permit Price'])
    outputs = list(df['Outputs'])
    emissions = list(df['Emissions'])
    profits = list(df['Profits'])
except:
    percentages = []
    caps2 = []
    permit_prices = []
    outputs = []
    emissions = []
    profits = []
# Loop through all the emission caps and calculate the permit price

# Find all the caps that are not calculated yet (meaning that they are more than 0.5 different than a value of the caps2)
for i in caps[:]:  # Iterate over a copy of the list
    if any(abs(i - x) < 0.5 for x in caps2):
        caps.remove(i)

for i in tqdm.tqdm(caps):
    Regulator1.emission_cap = i  
    Regulator1.find_optimal_permit_price_to_meet_the_emission_cap_requirements(precision= 0.1, x_high = 1000000000000)
    caps2.append(i)
    percentages.append(Regulator1.emission_cap/BAU_emissions)
    permit_prices.append(Regulator1.permit_price)
    outputs.append(tuple([firm.actual_output for firm in Regulator1.firm_registry.values()]))
    emissions.append(tuple([firm.emission for firm in Regulator1.firm_registry.values()]))
    profits.append(tuple([firm.profit for firm in Regulator1.firm_registry.values()]))
    # save all the data in a csv file
    df = pd.DataFrame({'Percentage of BAU': percentages, 'Emission Cap': caps2, 'Permit Price': permit_prices, 'Outputs': outputs, 'Emissions': emissions, 'Profits': profits})
    df.to_csv('permit_price_to_Percentage_of_BAU_v0_1.csv', index=False)




  0%|          | 0/20 [00:00<?, ?it/s]

Permit price: 3906250000.0000003906250000.0, cap = 448, second_stage = False, a = 1.000000

  0%|          | 0/20 [00:13<?, ?it/s]


KeyboardInterrupt: 

Εδώ θα τρέξει το ίδιο, αλ΄λα κοιτώντας μόνο το τελευταίο για να δει αν αποκλίνει. 

In [8]:
# Create a plot for the permit price with respect to the emission cap 10 samples between 0 and 200
number_of_samples = 100
max_allocation = 1
lowest_allocation = 0
BAU_percentage = np.linspace(lowest_allocation, max_allocation , number_of_samples)
BAU_emissions = Regulator1.BAU_emissions
caps = [BAU_emissions*i for i in BAU_percentage]
# Read the data from the csv file, if it exists, if not, just create a new one
try:
    df = pd.read_csv('../data/generated/permit_price_to_Percentage_of_BAU_v1_1.csv')
    percentages = list(df['Percentage of BAU'])
    caps2 = list(df['Emission Cap'])
    permit_prices = list(df['Permit Price'])
    outputs = list(df['Outputs'])
    emissions = list(df['Emissions'])
    profits = list(df['Profits'])
except:
    percentages = []
    caps2 = []
    permit_prices = []
    outputs = []
    emissions = []
    profits = []
# Loop through all the emission caps and calculate the permit price
print("caps{}".format(caps))
# Find all the caps that are not calculated yet (meaning that they are more than 0.5 different than a value of the caps2)
for i in caps[:]:  # Iterate over a copy of the list
    if any(abs(i - x) < 0.1 for x in caps2):
        caps.remove(i)

for i in tqdm.tqdm(caps):
    Regulator1.emission_cap = i  
    Regulator1.find_optimal_permit_price_to_meet_the_emission_cap_requirements(precision= 0.05, x_high = 1000, size_of_diffs= 5)
    caps2.append(i)
    percentages.append(Regulator1.emission_cap/BAU_emissions)
    permit_prices.append(Regulator1.permit_price)
    outputs.append(tuple([firm.actual_output for firm in Regulator1.firm_registry.values()]))
    emissions.append(tuple([firm.emission for firm in Regulator1.firm_registry.values()]))
    profits.append(tuple([firm.profit for firm in Regulator1.firm_registry.values()]))
    # save all the data in a csv file
    df = pd.DataFrame({'Percentage of BAU': percentages, 'Emission Cap': caps2, 'Permit Price': permit_prices, 'Outputs': outputs, 'Emissions': emissions, 'Profits': profits})
    df.to_csv('../data/generated/permit_price_to_Percentage_of_BAU_v1_1.csv', index=False)




caps[0.0, 22.630779076264965, 45.26155815252993, 67.89233722879489, 90.52311630505986, 113.15389538132483, 135.78467445758977, 158.41545353385476, 181.04623261011972, 203.67701168638467, 226.30779076264966, 248.93856983891462, 271.56934891517955, 294.20012799144456, 316.8309070677095, 339.4616861439745, 362.09246522023943, 384.72324429650445, 407.35402337276935, 429.9848024490343, 452.6155815252993, 475.2463606015642, 497.87713967782923, 520.5079187540942, 543.1386978303591, 565.7694769066242, 588.4002559828891, 611.0310350591541, 633.661814135419, 656.2925932116839, 678.923372287949, 701.5541513642139, 724.1849304404789, 746.8157095167438, 769.4464885930089, 792.0772676692739, 814.7080467455387, 837.3388258218037, 859.9696048980686, 882.6003839743337, 905.2311630505986, 927.8619421268636, 950.4927212031284, 973.1235002793935, 995.7542793556585, 1018.3850584319234, 1041.0158375081885, 1063.6466165844533, 1086.2773956607182, 1108.9081747369833, 1131.5389538132483, 1154.1697328895134, 11

  0%|          | 0/1 [00:00<?, ?it/s]

Max diff: 0.02, permit price = 29.3, cap = 1765, second_stage = False, a = 0.81000

100%|██████████| 1/1 [03:20<00:00, 200.29s/it]

Permit price: 29.296875 and total emission: 1764.9493015999308 and emission cap 1765.2007679486674
